In [1]:
import pandas as pd 
import os 
import numpy as np 
from pathlib import Path
from typing import *
from sklearn.linear_model import LinearRegression
import seaborn as sns

# Data Functions

In [2]:
ROOT = os.path.dirname(os.getcwd())+"/data"

def load(path:Path):
    return pd.read_csv(path, sep="|")

def collect_time_(df:pd.DataFrame,column:str):
    data_times = pd.to_datetime(df[column]).dt
    df["year"] = data_times.year
    df["month"] = data_times.month
    del df_meteo[column]

def set_pickle(object_to_save:object, path:Path):
    with open(path, "wb") as handle:
        pickle.dump(object_to_save, handle, protocol=pickle.HIGHEST_PROTOCOL)    

def get_pickle(path:Path):
    with open(path, "rb") as handle:
        o=pickle.load(handle)     
    return o


In [134]:
df_meteo = load(ROOT+"/DATOS_METEO.TXT") 
df_train = load(ROOT+"/UH_2023_TRAIN.txt")

In [135]:
df_train["SUPERFICIE"]

,CAMPAÑA,ID_FINCA,ID_ZONA,ID_ESTACION,ALTITUD,VARIEDAD,MODO,TIPO,COLOR,SUPERFICIE,PRODUCCION
0,14,76953,515,4,660,26,2,0,1,0.0000,22215.0
1,14,84318,515,4,660,26,2,0,1,0.0000,22215.0
2,14,85579,340,4,520,32,2,0,1,0.0000,20978.0
3,14,69671,340,4,520,32,2,0,1,0.0000,40722.0
4,14,14001,852,14,NaN,81,1,0,1,0.0000,14126.0
...,...,...,...,...,...,...,...,...,...,...,...
9596,22,37461,239,6,700,52,2,0,1,3.6800,NaN
9597,22,58769,239,6,700,32,2,0,1,4.2500,NaN
9598,22,58769,239,6,700,59,2,0,1,4.0700,NaN
9599,22,88928,239,6,700,40,2,0,1,4.5727,NaN


In [148]:
len(df_meteo["ID_ESTACION"].unique())

20

In [149]:
df_meteo["ID_ESTACION"]

0          13
1          13
2          13
3          13
4          13
           ..
1223655     8
1223656     8
1223657     8
1223658     8
1223659     8
Name: ID_ESTACION, Length: 1223660, dtype: int64

In [113]:
collect_time_(df_meteo,"validTimeUtc")

In [124]:
df_meteo.precip24Hour.value_counts()

0.0      908756
0.1       29588
0.2       19347
0.3       15495
0.4       11460
          ...  
88.5          1
111.2         1
117.6         1
103.9         1
82.0          1
Name: precip24Hour, Length: 1262, dtype: int64

In [115]:
## Creating new rows with the splited values of the altitud
df_train["ALTITUD"] = df_train["ALTITUD"].apply(lambda x: str(x).split("-"))
print(df_train.shape)
df_train = df_train.explode("ALTITUD")

(9601, 11)


In [116]:
set(train.loc[(train["VARIEDAD"] == 81) & (train["CAMPAÑA"] != 22), "ALTITUD"].to_numpy())

{'390',
 '440',
 '450',
 '455',
 '460',
 '465',
 '470',
 '480',
 '500',
 '510',
 '520',
 '530',
 '540',
 '550',
 '570',
 '580',
 '600',
 '605',
 '610',
 '615',
 '620',
 '625',
 '630',
 '635',
 '640',
 '650',
 '660',
 '670',
 '680',
 '690',
 '700',
 '720',
 'nan'}

In [119]:
tmp = df_train[["ID_FINCA","ALTITUD"]].dropna();
dic_alt = dict(zip(tmp["ID_FINCA"], tmp["ALTITUD"]))
test = df_train.fillna(df_train["ID_FINCA"].map(dic_alt))

display(df_train[df_train["ID_ZONA"] == 292])
display(df_train[df_train["ID_FINCA"] == 83458])
display(test[test["VARIEDAD"] == 26]["ALTITUD"].unique())

,CAMPAÑA,ID_FINCA,ID_ZONA,ID_ESTACION,ALTITUD,VARIEDAD,MODO,TIPO,COLOR,SUPERFICIE,PRODUCCION
3934,17,9086,292,12,nan,26,2,0,1,0.0000,3590.000
3935,17,9086,292,12,nan,52,2,0,1,0.0000,4880.000
5003,18,83458,292,3,nan,10,2,0,1,0.0000,10490.000
5004,18,83458,292,3,nan,26,2,0,1,0.0000,6279.000
5005,18,9086,292,12,nan,26,2,0,1,0.0000,3381.000
5006,18,9086,292,12,nan,52,2,0,1,0.0000,10010.000
6073,19,83458,292,3,nan,10,2,0,1,0.0000,9030.000
6074,19,83458,292,3,nan,26,2,0,1,0.0000,7304.630
6075,19,9086,292,12,nan,26,2,0,1,0.0000,5635.370
6076,19,9086,292,12,nan,52,2,0,1,0.0000,3970.000


,CAMPAÑA,ID_FINCA,ID_ZONA,ID_ESTACION,ALTITUD,VARIEDAD,MODO,TIPO,COLOR,SUPERFICIE,PRODUCCION
5003,18,83458,292,3,nan,10,2,0,1,0.0000,10490.000
5004,18,83458,292,3,nan,26,2,0,1,0.0000,6279.000
6073,19,83458,292,3,nan,10,2,0,1,0.0000,9030.000
6074,19,83458,292,3,nan,26,2,0,1,0.0000,7304.630
7100,20,83458,292,3,nan,10,2,0,1,1.8125,7760.000
7101,20,83458,292,3,nan,26,2,0,1,1.0167,2523.315
8119,21,83458,292,3,nan,10,2,0,1,1.8125,16890.000
8120,21,83458,292,3,nan,26,2,0,1,1.0167,12125.460
9172,22,83458,292,3,nan,10,2,0,1,1.8125,NaN
9173,22,83458,292,3,nan,26,2,0,1,1.0167,NaN


array(['660', '600', '620', '630', '640', '560', '670', 'nan', '520',
       '540', '480', '610', '450'], dtype=object)

In [30]:
df_train[["ID_FINCA", "ALTITUD"]].value_counts()

ID_FINCA  ALTITUD
4024      650-660    59
49636     450-480    54
48827     550-610    47
84174     450-480    46
61177     600-610    36
                     ..
56332     620-640     1
23530     650-660     1
23348     455-465     1
80886     730         1
86768     520-530     1
Length: 1216, dtype: int64

In [16]:
df_train[["ID_FINCA", "ALTITUD"]].dtypes

ID_FINCA     int64
ALTITUD     object
dtype: object

In [7]:
df_train["VARIEDAD"].unique()

array([26, 32, 59, 40, 87, 17, 52, 81, 68,  9, 71, 15, 84, 62,  8, 43, 65,
       92, 55,  4, 94, 23, 38, 48])

# ETO

In [133]:
df_eto = load(ROOT+"/DATOS_ETO.TXT")
df_eto[df_eto.isna().any(axis=1)]


,date,DewpointLocalAfternoonAvg,DewpointLocalAfternoonMax,DewpointLocalAfternoonMin,DewpointLocalDayAvg,DewpointLocalDayMax,DewpointLocalDayMin,DewpointLocalDaytimeAvg,DewpointLocalDaytimeMax,DewpointLocalDaytimeMin,...,WindSpeedLocalMorningAvg,WindSpeedLocalMorningMax,WindSpeedLocalMorningMin,WindSpeedLocalNighttimeAvg,WindSpeedLocalNighttimeMax,WindSpeedLocalNighttimeMin,WindSpeedLocalOvernightAvg,WindSpeedLocalOvernightMax,WindSpeedLocalOvernightMin,ID_ESTACION
0,20150629,285.9,285.9,285.9,286.0,287.0,285.4,285.9,285.9,285.9,...,NaN,NaN,NaN,2.6,5.0,1.1,1.7,2.1,1.1,13
1,20150630,283.0,283.6,282.5,284.3,286.5,282.5,283.2,283.9,282.5,...,2.2,3.8,1.4,2.7,5.2,1.4,1.5,1.9,1.4,13
2,20150701,286.1,286.5,285.5,285.8,288.0,283.8,285.4,286.5,283.8,...,2.7,4.3,1.2,3.0,5.7,1.4,1.9,2.8,1.4,13
3,20150702,288.9,289.4,287.9,288.3,291.8,286.1,287.7,289.4,286.1,...,1.7,2.3,1.2,3.6,5.4,2.4,2.7,3.4,2.4,13
4,20150703,289.1,290.4,288.5,290.5,293.0,287.8,290.6,293.0,288.5,...,3.5,5.0,2.3,3.1,6.0,1.5,1.9,2.2,1.5,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51175,20220626,284.0,285.0,283.0,285.0,288.0,283.0,285.0,286.0,283.0,...,2.1,2.6,1.2,2.1,5.2,0.6,1.1,1.6,0.6,8
51176,20220627,289.0,289.0,288.0,288.0,290.0,286.0,289.0,290.0,288.0,...,3.2,3.9,2.2,3.1,5.0,1.4,1.8,2.8,1.4,8
51177,20220628,283.0,285.0,282.0,286.0,290.0,282.0,284.0,287.0,282.0,...,1.8,2.5,1.2,2.4,5.0,1.0,1.3,1.7,1.0,8
51178,20220629,283.0,285.0,282.0,285.0,288.0,282.0,284.0,286.0,282.0,...,2.2,3.1,1.3,2.7,5.4,1.3,1.6,1.9,1.3,8


In [143]:
for i in df_eto.columns:
    print(i)

date
DewpointLocalAfternoonAvg
DewpointLocalAfternoonMax
DewpointLocalAfternoonMin
DewpointLocalDayAvg
DewpointLocalDayMax
DewpointLocalDayMin
DewpointLocalDaytimeAvg
DewpointLocalDaytimeMax
DewpointLocalDaytimeMin
DewpointLocalEveningAvg
DewpointLocalEveningMax
DewpointLocalEveningMin
DewpointLocalMorningAvg
DewpointLocalMorningMax
DewpointLocalMorningMin
DewpointLocalNighttimeAvg
DewpointLocalNighttimeMax
DewpointLocalNighttimeMin
DewpointLocalOvernightAvg
DewpointLocalOvernightMax
DewpointLocalOvernightMin
EvapotranspirationLocalAfternoonAvg
EvapotranspirationLocalAfternoonMax
EvapotranspirationLocalAfternoonMin
EvapotranspirationLocalDayAvg
EvapotranspirationLocalDayMax
EvapotranspirationLocalDayMin
EvapotranspirationLocalDaytimeAvg
EvapotranspirationLocalDaytimeMax
EvapotranspirationLocalDaytimeMin
EvapotranspirationLocalEveningAvg
EvapotranspirationLocalEveningMax
EvapotranspirationLocalEveningMin
EvapotranspirationLocalMorningAvg
EvapotranspirationLocalMorningMax
Evapotranspirat

,date,PrecipAmountLocalDayAvg
0,20150629,0.0
1,20150630,0.0
2,20150701,0.0
3,20150702,0.0
4,20150703,0.0
...,...,...
51175,20220626,0.0
51176,20220627,0.0
51177,20220628,0.0
51178,20220629,0.0


# Explore Functions

In [22]:
   # In this case the test will be the year 2022
# TODO MIRAR MILLOR EL JOIN
def join_train_test_split(df1:pd.DataFrame, df2:pd.DataFrame):

    df1 = df2.join(df1, on="ID_ESTACION",lsuffix="_" ,how="outer")

    train = df1[df1["year"] != 2022]
    test = df1[df1["year"] == 2022]

    train_y = train["PRODUCCION"]
    test_y = test["PRODUCCION"]

    train.drop("PRODUCCION", axis="columns")
    test.drop("PRODUCCION", axis="columns")

    return train, test, train_y, test_y 

    
def compute_correlations(df:pd.DataFrame):
    return (df.corr().abs().unstack().sort_values(ascending=False).drop_duplicates())

def topk_correlations(df:pd.DataFrame, k:float=0.7):
    correlations = compute_correlations(df)
    
    top_k_percent = [pairs for pairs, value in correlations.items() if value >= k]
    return top_k_percent

def select_features_LR(X_train, y_train,X_test, y_test, correlation_pairs):
    pairs_scores = []
    features1,features2 = correlation_pairs
    
    #train
    y_train_indexes = y_train.index[y_train.isna() == False]
    y_train = y_train.dropna()
    
    #test
    y_test_indexes = y_test.index[y_test.isna() == False]
    y_test = y_test.dropna()
    
    
    for p1, p2 in zip(features1,features2):
        #fit the regressors
        Xtr = X_train.iloc[y_train_indexes]
        Xt = X_test.iloc[y_test_indexes]
        
        reg1 = LinearRegression().fit(Xtr[p1].to_numpy().reshape(-1,1), y_train)
        reg2 = LinearRegression().fit(Xtr[p2].to_numpy().reshape(-1,1), y_train)
        
        #scores
        s1 = reg1.score(Xt[p1].to_numpy().reshape(-1,1), y_test)
        s2 = reg2.score(Xt[p2].to_numpy().reshape(-1,1), y_test)
        
        if s1 >= s2:pairs_scores.append((p1,s1, f"lose {p2}"))
            
        else:pairs_scores.append((p2,s2, f"lose {p1}"))
            
        
    return pairs_scores



In [13]:
X_train, X_test, y_train, y_test = join_train_test_split(df_meteo,df_train)


In [14]:
X_train["VARIEDAD"].unique()

array([26., 32., 59., 40., 87., 17., 52., 81., 68.,  9.,  8., 94., 71.,
       15., 23., 43., 48., 62., 65.,  4., 84., 55., 92., 38., nan])

In [134]:
X_test.shape, y_test.shape

((9814, 45), (9814,))

In [135]:
correlations = list(zip(*topk_correlations(X_train)))

In [136]:
newX_train = X_train[X_train["year"] != 2021].reset_index(drop=True)
newy_train = y_train[X_train["year"] != 2021].reset_index(drop=True)

newX_test = X_train[X_train["year"] == 2021].reset_index(drop=True)
newy_test = y_train[X_train["year"] == 2021].reset_index(drop=True)



In [137]:
selected = select_features_LR(X_train=newX_train, y_train=newy_train, X_test=newX_test, y_test=newy_test, correlation_pairs=correlations)
(selected)

[('precip1Hour', -2.8779504330556094e-05, 'lose precip1Hour'),
 ('snow3Day', -2.0864610211557277e-05, 'lose snow2Day'),
 ('temperatureFeelsLike', 0.0010773450963691422, 'lose temperature'),
 ('ID_ZONA', 0.0041460349189286205, 'lose ID_ESTACION'),
 ('ID_ESTACION', -6.263131300543456e-05, 'lose ID_ESTACION_train'),
 ('snow2Day', -2.2035554688493164e-05, 'lose snow24Hour'),
 ('snow3Day', -2.0864610211557277e-05, 'lose snow24Hour'),
 ('precip3Day', -1.0347280501310507e-05, 'lose precip2Day'),
 ('snow3Day', -2.0864610211557277e-05, 'lose snow7Day'),
 ('snow7Day', -2.1595243673511177e-05, 'lose snow2Day'),
 ('windSpeed', 9.527136916642753e-05, 'lose windGust'),
 ('ID_ZONA', 0.0041460349189286205, 'lose ID_ESTACION_train'),
 ('precipYtd', 8.137255169604618e-05, 'lose month'),
 ('precip24Hour', -1.0936298537655986e-05, 'lose precip2Day'),
 ('temperature', 0.0009253992479907325, 'lose temperatureMax24Hour'),
 ('temperatureFeelsLike', 0.0010773450963691422, 'lose temperatureMax24Hour'),
 ('snow7

# Plot Functions


In [185]:
def plot_distribution(df:pd.DataFrame, column:str):
    if df.get("year", None) is not None:
        sns.displot(df, x=column, hue="year", multiple="dodge")
    else:
        sns.displot(df, x=column)
        

def plot_forecasting(x_axis, y_axis, classes: Optional[list], column:str):
    sns.relplot(x=x_axis,y=y_axis,kind="line", hue=classes, markers=True,legend=True)

In [234]:
X_train.columns

Index(['CAMPAÑA', 'ID_FINCA', 'ID_ZONA', 'ID_ESTACION_train', 'ALTITUD',
       'VARIEDAD', 'MODO', 'TIPO', 'COLOR', 'SUPERFICIE', 'PRODUCCION',
       'precip1Hour', 'precip6Hour', 'precip24Hour', 'precip2Day',
       'precip3Day', 'precip7Day', 'precipMtd', 'precipYtd', 'pressureChange',
       'pressureMeanSeaLevel', 'relativeHumidity', 'snow1Hour', 'snow6Hour',
       'snow24Hour', 'snow2Day', 'snow3Day', 'snow7Day', 'snowMtd',
       'snowSeason', 'snowYtd', 'temperature', 'temperatureChange24Hour',
       'temperatureMax24Hour', 'temperatureMin24Hour', 'temperatureDewPoint',
       'temperatureFeelsLike', 'uvIndex', 'visibility', 'windDirection',
       'windGust', 'windSpeed', 'ID_ESTACION', 'year', 'month'],
      dtype='object')

In [16]:
X_train.groupby(["VARIEDAD", "ID_ESTACION_"]).mean()

CAMPAÑA      ID_FINCA     ID_ZONA  MODO  TIPO  COLOR  \
VARIEDAD ID_ESTACION_                                                           
4.0      5.0           18.280000  50081.640000  340.040000   2.0   1.0    1.0   
         16.0          18.000000  44057.000000  272.000000   2.0   1.0    1.0   
8.0      3.0           19.500000  59388.000000   71.000000   1.5   0.0    0.0   
         7.0           19.000000  64798.000000  616.000000   2.0   0.0    0.0   
         15.0          17.500000  52008.000000  412.000000   2.0   0.0    0.0   
...                          ...           ...         ...   ...   ...    ...   
94.0     9.0           20.500000  13333.000000  784.000000   2.0   0.0    0.0   
         15.0          19.562500  49373.625000  464.250000   2.0   0.0    0.0   
         16.0          20.000000  53738.750000  772.500000   2.0   0.0    0.0   
         18.0          20.750000  66942.125000  277.625000   2.0   0.0    0.0   
         19.0          20.666667  61622.333333  557.333333   2.0   0.0    0.0   

                       SUPERFICIE    PRODUCCION  precip1Hour  precip6Hour  \
VARIEDAD ID_ESTACION_                                                       
4.0      5.0             2.907208  19812.412640          NaN          NaN   
         16.0            1.158571  14672.834571          NaN          NaN   
8.0      3.0             1.181950  12205.000000          NaN          NaN   
         7.0             0.629400   3789.000000          NaN          NaN   
         15.0            1.776950  37654.566000          NaN          NaN   
...                           ...           ...          ...          ...   
94.0     9.0             1.459700   8970.000000          NaN          NaN   
         15.0            1.541813  11294.548000          NaN          NaN   
         16.0            2.245800  10133.750000          NaN          NaN   
         18.0            3.057438   9850.904000          NaN          NaN   
         19.0            1.340567   5146.666667          NaN          NaN   

                       ...  temperatureDewPoint  temperatureFeelsLike  \
VARIEDAD ID_ESTACION_  ...                                              
4.0      5.0           ...                  NaN                   NaN   
         16.0          ...                  NaN                   NaN   
8.0      3.0           ...                  NaN                   NaN   
         7.0           ...                  NaN                   NaN   
         15.0          ...                  NaN                   NaN   
...                    ...                  ...                   ...   
94.0     9.0           ...                  NaN                   NaN   
         15.0          ...                  NaN                   NaN   
         16.0          ...                  NaN                   NaN   
         18.0          ...                  NaN                   NaN   
         19.0          ...                  NaN                   NaN   

                       uvIndex  visibility  windDirection  windGust  \
VARIEDAD ID_ESTACION_                                                 
4.0      5.0               NaN         NaN            NaN       NaN   
         16.0              NaN         NaN            NaN       NaN   
8.0      3.0               NaN         NaN            NaN       NaN   
         7.0               NaN         NaN            NaN       NaN   
         15.0              NaN         NaN            NaN       NaN   
...                        ...         ...            ...       ...   
94.0     9.0               NaN         NaN            NaN       NaN   
         15.0              NaN         NaN            NaN       NaN   
         16.0              NaN         NaN            NaN       NaN   
         18.0              NaN         NaN            NaN       NaN   
         19.0              NaN         NaN            NaN       NaN   

                       windSpeed  ID_ESTACION  year  month  
VARIEDAD ID_ESTACION_             

In [232]:
(df_train["VARIEDAD"].unique())

array([26, 32, 59, 40, 87, 17, 52, 81, 68,  9, 71, 15, 84, 62,  8, 43, 65,
       92, 55,  4, 94, 23, 38, 48])

In [21]:
X_train

,CAMPAÑA,ID_FINCA,ID_ZONA,ID_ESTACION_,ALTITUD,VARIEDAD,MODO,TIPO,COLOR,SUPERFICIE,...,temperatureDewPoint,temperatureFeelsLike,uvIndex,visibility,windDirection,windGust,windSpeed,ID_ESTACION,year,month
0.0,14.0,76953.0,515.0,4.0,660,26.0,2.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,NaN
1.0,14.0,84318.0,515.0,4.0,660,26.0,2.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,NaN
2.0,14.0,85579.0,340.0,4.0,520,32.0,2.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,NaN
3.0,14.0,69671.0,340.0,4.0,520,32.0,2.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,NaN
12.0,14.0,5696.0,919.0,14.0,650-660,59.0,1.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.1,9.5,0.0,13.82,300.0,34.6,21.6,1219242,2021.0,12.0
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.1,9.0,0.0,13.98,290.0,30.2,19.4,1219243,2021.0,12.0
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.8,8.8,0.0,14.15,300.0,29.9,19.4,1219244,2021.0,12.0
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.0,8.0,0.0,13.55,290.0,28.4,19.1,1219247,2021.0,12.0


In [18]:
(X_train.groupby(["ID_FINCA", "VARIEDAD", "year"])).mean()

,,,CAMPAÑA,ID_ZONA,ID_ESTACION_,MODO,TIPO,COLOR,SUPERFICIE,PRODUCCION,precip1Hour,precip6Hour,...,temperatureMin24Hour,temperatureDewPoint,temperatureFeelsLike,uvIndex,visibility,windDirection,windGust,windSpeed,ID_ESTACION,month
ID_FINCA,VARIEDAD,year,,,,,,,,,,,,,,,,,,,,,
